# PERKENALAN

NAMA : MUHAMMAD HANIF SUYUTHI

PROGRAM : FTDS

BATCH : RMT-019

# DEFAULT RISK - HOME DEPOT

# DESKRIPSI

## OBJEKTIF
MEMBUAT DEFAULT PREDICTION MODEL

## PERMASALAHAN
- APA SAJA FITUR YANG MEMPENGARUHI DEFAULT
- BAGAIMANAKAH HASIL DARI MODEL PREDIKSI YANG KITA BUAT


## DATASET
https://www.kaggle.com/competitions/home-credit-default-risk/data


In [ ]:
pip install feature_engine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Impor Pustaka

In [130]:
import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

from scipy.stats import kendalltau
from scipy.stats import chi2_contingency

from scipy.stats import skew


from feature_engine.outliers import Winsorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier

import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
#from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

#save model
import json
import pickle

import warnings
warnings.filterwarnings("ignore")

In [131]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Memuat Data

In [132]:
df1=pd.read_csv("/content/drive/MyDrive/Hacktiv8/Final Project/application_train.csv")
df2=pd.read_csv("/content/drive/MyDrive/Hacktiv8/Final Project/application_test.csv")

In [133]:
df1.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
df6=pd.read_csv("/content/drive/MyDrive/Hacktiv8/Final Project/HomeCredit_columns_description.csv",encoding='latin-1')

In [135]:
df6

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...,...
214,217,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
215,218,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
216,219,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
217,220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN


In [136]:
df_desc=df6[df6['Table']=='application_{train|test}.csv']

In [137]:
df_desc

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...,...
117,120,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,NaN
118,121,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,NaN
119,122,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,NaN
120,123,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the...,NaN


In [138]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 122 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    SK_ID_CURR                    int64  
 1    TARGET                        int64  
 2    NAME_CONTRACT_TYPE            object 
 3    CODE_GENDER                   object 
 4    FLAG_OWN_CAR                  object 
 5    FLAG_OWN_REALTY               object 
 6    CNT_CHILDREN                  int64  
 7    AMT_INCOME_TOTAL              float64
 8    AMT_CREDIT                    float64
 9    AMT_ANNUITY                   float64
 10   AMT_GOODS_PRICE               float64
 11   NAME_TYPE_SUITE               object 
 12   NAME_INCOME_TYPE              object 
 13   NAME_EDUCATION_TYPE           object 
 14   NAME_FAMILY_STATUS            object 
 15   NAME_HOUSING_TYPE             object 
 16   REGION_POPULATION_RELATIVE    float64
 17   DAYS_BIRTH                    int64  
 18   DA

In [139]:
df1.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


# Fitur Engineering

In [140]:
cat_cols=df1.select_dtypes(include='object').columns.tolist()
num_cols=df1.select_dtypes(include=['int','float']).columns.tolist()

In [141]:
data = {
    'fitur': df1[cat_cols].columns,
    'nuniq': df1[cat_cols].nunique(),
    'uniq': df1[cat_cols].apply(lambda x: x.unique())
}
df_uniq = pd.DataFrame(data)

df_uniq

,fitur,nuniq,uniq
NAME_CONTRACT_TYPE,NAME_CONTRACT_TYPE,2,"[Cash loans, Revolving loans]"
CODE_GENDER,CODE_GENDER,3,"[M, F, XNA]"
FLAG_OWN_CAR,FLAG_OWN_CAR,2,"[N, Y]"
FLAG_OWN_REALTY,FLAG_OWN_REALTY,2,"[Y, N]"
NAME_TYPE_SUITE,NAME_TYPE_SUITE,7,"[Unaccompanied, Family, Spouse, partner, Child..."
NAME_INCOME_TYPE,NAME_INCOME_TYPE,8,"[Working, State servant, Commercial associate,..."
NAME_EDUCATION_TYPE,NAME_EDUCATION_TYPE,5,"[Secondary / secondary special, Higher educati..."
NAME_FAMILY_STATUS,NAME_FAMILY_STATUS,6,"[Single / not married, Married, Civil marriage..."
NAME_HOUSING_TYPE,NAME_HOUSING_TYPE,6,"[House / apartment, Rented apartment, With par..."
OCCUPATION_TYPE,OCCUPATION_TYPE,18,"[Laborers, Core staff, Accountants, Managers, ..."


In [142]:
missing_values = df1.isnull().sum()

missing_percentage = (missing_values / len(df1)) * 100

missing_df = pd.DataFrame({'kolom': missing_values.index, '%missing value': missing_percentage.values})

print(missing_df)

                          kolom  %missing value
0                    SK_ID_CURR        0.000000
1                        TARGET        0.000000
2            NAME_CONTRACT_TYPE        0.000000
3                   CODE_GENDER        0.000000
4                  FLAG_OWN_CAR        0.000000
..                          ...             ...
117   AMT_REQ_CREDIT_BUREAU_DAY       13.501631
118  AMT_REQ_CREDIT_BUREAU_WEEK       13.501631
119   AMT_REQ_CREDIT_BUREAU_MON       13.501631
120   AMT_REQ_CREDIT_BUREAU_QRT       13.501631
121  AMT_REQ_CREDIT_BUREAU_YEAR       13.501631

[122 rows x 2 columns]


In [143]:
sorted_missing_df = missing_df.sort_values(by='%missing value', ascending=False)
sorted_missing_df

,kolom,%missing value
76,COMMONAREA_MEDI,69.872297
48,COMMONAREA_AVG,69.872297
62,COMMONAREA_MODE,69.872297
70,NONLIVINGAPARTMENTS_MODE,69.432963
56,NONLIVINGAPARTMENTS_AVG,69.432963
...,...,...
15,NAME_HOUSING_TYPE,0.000000
14,NAME_FAMILY_STATUS,0.000000
13,NAME_EDUCATION_TYPE,0.000000
12,NAME_INCOME_TYPE,0.000000


Terlihat bahwa banyak kolom yang memiliki persen Nan > 40%

Sealnjutnya kita akan melihat dan menangani kolom yang memiliki nilai yang ganjil

In [144]:
df_de=df1[df1['CODE_GENDER'] == "XNA"]
df_de['CODE_GENDER']

35657     XNA
38566     XNA
83382     XNA
189640    XNA
Name: CODE_GENDER, dtype: object

Terdapat nilai XNA pada kolom code gender. Terlihat bahwa nilai XNA hanya ada 4 data, sehingga nilai ini akan kita hapus.

In [145]:
df1['DAYS_EMPLOYED'].describe()

count    307511.000000
mean      63815.045904
std      141275.766519
min      -17912.000000
25%       -2760.000000
50%       -1213.000000
75%        -289.000000
max      365243.000000
Name: DAYS_EMPLOYED, dtype: float64

In [146]:
df_de=df1[df1['DAYS_EMPLOYED'] > 0.0]
df_de['DAYS_EMPLOYED']

8         365243
11        365243
23        365243
38        365243
43        365243
           ...  
307469    365243
307483    365243
307487    365243
307505    365243
307507    365243
Name: DAYS_EMPLOYED, Length: 55374, dtype: int64

Terlihat bahwa nilai pada kolon days employed kebanyakan bernilai negatif dan hanya ada 1 nilai yang positif

In [147]:
df1['AMT_INCOME_TOTAL'].describe()

count    3.075110e+05
mean     1.687979e+05
std      2.371231e+05
min      2.565000e+04
25%      1.125000e+05
50%      1.471500e+05
75%      2.025000e+05
max      1.170000e+08
Name: AMT_INCOME_TOTAL, dtype: float64

Pada kolom amt income total terlihat ada nilai yang terlalu besar. Nilai ini akan kita hapus

In [29]:
df1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
SK_ID_CURR,307511.0,278180.518577,102790.175348,100002.0,189145.5,278202.0,367142.5,456255.0
TARGET,307511.0,0.080729,0.272419,0.0,0.0,0.0,0.0,1.0
CNT_CHILDREN,307511.0,0.417052,0.722121,0.0,0.0,0.0,1.0,19.0
AMT_INCOME_TOTAL,307511.0,168797.919297,237123.146279,25650.0,112500.0,147150.0,202500.0,117000000.0
AMT_CREDIT,307511.0,599025.999706,402490.776996,45000.0,270000.0,513531.0,808650.0,4050000.0
...,...,...,...,...,...,...,...,...
AMT_REQ_CREDIT_BUREAU_DAY,265992.0,0.007000,0.110757,0.0,0.0,0.0,0.0,9.0
AMT_REQ_CREDIT_BUREAU_WEEK,265992.0,0.034362,0.204685,0.0,0.0,0.0,0.0,8.0
AMT_REQ_CREDIT_BUREAU_MON,265992.0,0.267395,0.916002,0.0,0.0,0.0,0.0,27.0
AMT_REQ_CREDIT_BUREAU_QRT,265992.0,0.265474,0.794056,0.0,0.0,0.0,0.0,261.0


In [30]:
df1 = df1[df1['CODE_GENDER']!='XNA']

df1['CODE_GENDER'].value_counts()

F    202448
M    105059
Name: CODE_GENDER, dtype: int64

In [31]:
df1 = df1[df1['AMT_INCOME_TOTAL'] != 117000000.0]
df1['AMT_INCOME_TOTAL'].describe()

count    3.075060e+05
mean     1.684178e+05
std      1.088124e+05
min      2.565000e+04
25%      1.125000e+05
50%      1.471500e+05
75%      2.025000e+05
max      1.800009e+07
Name: AMT_INCOME_TOTAL, dtype: float64

In [32]:
df1['DAYS_EMPLOYED'] = df1['DAYS_EMPLOYED'].apply(lambda x: np.nan if x==365243 else x)
df2['DAYS_EMPLOYED'] = df2['DAYS_EMPLOYED'].apply(lambda x: np.nan if x==365243 else x)

Nilai numerik yang semuanya memiliki nilai negatif kita ubah menjadi nilai positif

In [33]:
df1[num_cols]=df1[num_cols].abs()

Kemudian kita akan menghapus kolom yang memiliki nilai missing value

In [34]:
threshold = len(df1) * 0.6
df1_cleaned = df1.dropna(thresh=threshold, axis=1)
data=df1_cleaned

In [35]:
data.shape

(307506, 73)

In [36]:
# Menghitung jumlah missing value untuk setiap kolom
missing_values = df1_cleaned.isnull().sum()

# Menghitung persentase missing value untuk setiap kolom
missing_percentage = (missing_values / len(df1_cleaned)) * 100

# Membuat DataFrame untuk missing value
missing_df = pd.DataFrame({'kolom': missing_values.index, '%missing value': missing_percentage.values})

# Menampilkan hasil
missing_df

,kolom,%missing value
0,SK_ID_CURR,0.00000
1,TARGET,0.00000
2,NAME_CONTRACT_TYPE,0.00000
3,CODE_GENDER,0.00000
4,FLAG_OWN_CAR,0.00000
...,...,...
68,AMT_REQ_CREDIT_BUREAU_DAY,13.50185
69,AMT_REQ_CREDIT_BUREAU_WEEK,13.50185
70,AMT_REQ_CREDIT_BUREAU_MON,13.50185
71,AMT_REQ_CREDIT_BUREAU_QRT,13.50185


## Bagi Data

In [37]:
data=df1

In [38]:
X = data.drop(['TARGET'], axis=1)
y = data['TARGET']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=20)

In [40]:
X_train.describe()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,246004.000000,246004.000000,2.460040e+05,2.460040e+05,245993.000000,2.457830e+05,246004.000000,246004.000000,201599.000000,246004.000000,...,246004.000000,246004.000000,246004.000000,246004.000000,212822.000000,212822.000000,212822.000000,212822.000000,212822.000000,212822.000000
mean,278341.672717,0.417383,1.683439e+05,5.990907e+05,27091.758983,5.383926e+05,0.020877,16043.055336,2384.716799,4985.111120,...,0.008175,0.000622,0.000512,0.000341,0.006306,0.007062,0.034324,0.267543,0.266345,1.899052
std,102860.483362,0.723205,1.110738e+05,4.025138e+05,14474.155584,3.693229e+05,0.013835,4362.732622,2337.924961,3522.911264,...,0.090044,0.024931,0.022626,0.018475,0.083435,0.110453,0.205280,0.915601,0.832984,1.868297
min,100002.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.500000e+04,0.000290,7489.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189249.750000,0.000000,1.125000e+05,2.700000e+05,16515.000000,2.385000e+05,0.010006,12425.000000,767.000000,2008.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278432.000000,0.000000,1.485000e+05,5.130405e+05,24885.000000,4.500000e+05,0.018850,15759.500000,1649.000000,4504.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367358.250000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,19684.250000,3174.500000,7479.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456253.000000,19.000000,1.800009e+07,4.050000e+06,258025.500000,4.050000e+06,0.072508,25229.000000,17912.000000,24672.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


## Seleksi Fitur

In [41]:
cat_cols=X_train.select_dtypes(include='object').columns.tolist()
num_cols=X_train.select_dtypes(include=['int','float']).columns.tolist()

In [42]:
df_korelasi_cat = pd.DataFrame(columns=['Kolom Kategorik', 'Chi-Square', 'P-value'])

#Mencari nilai chi
for kolom in cat_cols:
    cross_tab = pd.crosstab(X_train[kolom], y)
    chi2, p_value, _, _ = chi2_contingency(cross_tab)
    new_row = pd.DataFrame({'Kolom Kategorik': [kolom], 'Chi-Square': [chi2], 'P-value': [p_value]})
    df_korelasi_cat = pd.concat([df_korelasi_cat, new_row], ignore_index=True)

In [43]:
df_korelasi_cat

,Kolom Kategorik,Chi-Square,P-value
0,NAME_CONTRACT_TYPE,232.526458,1.676398e-52
1,CODE_GENDER,739.083398,9.486082e-163
2,FLAG_OWN_CAR,102.658285,3.982425e-24
3,FLAG_OWN_REALTY,11.576002,6.680843e-04
4,NAME_TYPE_SUITE,31.012249,2.521314e-05
5,NAME_INCOME_TYPE,1047.138602,7.846495e-222
6,NAME_EDUCATION_TYPE,840.624177,1.217259e-180
7,NAME_FAMILY_STATUS,446.481807,2.819336e-94
8,NAME_HOUSING_TYPE,347.426432,6.268542e-73
9,OCCUPATION_TYPE,1163.760509,7.714339e-237


In [44]:
filtered_df = df_korelasi_cat[(df_korelasi_cat['P-value'] < 0.05)]
filtered_df = filtered_df[['Kolom Kategorik', 'Chi-Square', 'P-value']]

In [45]:
filtered_df

,Kolom Kategorik,Chi-Square,P-value
0,NAME_CONTRACT_TYPE,232.526458,1.676398e-52
1,CODE_GENDER,739.083398,9.486082e-163
2,FLAG_OWN_CAR,102.658285,3.982425e-24
3,FLAG_OWN_REALTY,11.576002,6.680843e-04
4,NAME_TYPE_SUITE,31.012249,2.521314e-05
5,NAME_INCOME_TYPE,1047.138602,7.846495e-222
6,NAME_EDUCATION_TYPE,840.624177,1.217259e-180
7,NAME_FAMILY_STATUS,446.481807,2.819336e-94
8,NAME_HOUSING_TYPE,347.426432,6.268542e-73
9,OCCUPATION_TYPE,1163.760509,7.714339e-237


In [46]:
cat=filtered_df['Kolom Kategorik'].tolist()

In [47]:
cat

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [48]:
df_korelasi_num = pd.DataFrame(columns=['Kolom Numerik', 'pearson', 'spearman'])

# korelasi kolom numerik
for kolom in num_cols:
    korelasi_pearson = X_train[kolom].corr(data['TARGET'])
    korelasi_spearman = X_train[kolom].corr(data['TARGET'], method='spearman')
    new_row = pd.DataFrame({'Kolom Numerik': [kolom], 'pearson': [korelasi_pearson], 'spearman': [korelasi_spearman]})
    df_korelasi_num = pd.concat([df_korelasi_num, new_row], ignore_index=True)

In [49]:
df_korelasi_num

,Kolom Numerik,pearson,spearman
0,SK_ID_CURR,-0.001773,-0.001778
1,CNT_CHILDREN,0.019982,0.020684
2,AMT_INCOME_TOTAL,-0.019363,-0.017574
3,AMT_CREDIT,-0.030687,-0.017772
4,AMT_ANNUITY,-0.012875,-0.000229
...,...,...,...
100,AMT_REQ_CREDIT_BUREAU_DAY,0.001688,0.003631
101,AMT_REQ_CREDIT_BUREAU_WEEK,0.000513,0.000139
102,AMT_REQ_CREDIT_BUREAU_MON,-0.011005,-0.005777
103,AMT_REQ_CREDIT_BUREAU_QRT,-0.002890,-0.008954


In [50]:
filtered_df = df_korelasi_num[(df_korelasi_num['pearson'].abs() > 0.05) | (df_korelasi_num['spearman'].abs() > 0.05)]
filtered_df = filtered_df[['Kolom Numerik', 'pearson', 'spearman']]
print(filtered_df)

                  Kolom Numerik   pearson  spearman
7                    DAYS_BIRTH -0.078714 -0.078755
8                 DAYS_EMPLOYED -0.074810 -0.080693
10              DAYS_ID_PUBLISH -0.050684 -0.051733
11                  OWN_CAR_AGE  0.039734  0.054078
19         REGION_RATING_CLIENT  0.059420  0.059475
20  REGION_RATING_CLIENT_W_CITY  0.061428  0.061506
26       REG_CITY_NOT_WORK_CITY  0.052831  0.052831
28                 EXT_SOURCE_1 -0.155359 -0.151253
29                 EXT_SOURCE_2 -0.160599 -0.147541
30                 EXT_SOURCE_3 -0.177716 -0.165239
78       DAYS_LAST_PHONE_CHANGE -0.055684 -0.054245


In [51]:
num=filtered_df['Kolom Numerik'].tolist()

In [52]:
kolom_fitur = num + cat

In [53]:
X_train_fix=X_train[kolom_fitur]

In [54]:
X_train_fix

,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,OWN_CAR_AGE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,...,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
227344,9705,267.0,2311,NaN,2,2,0,NaN,0.047090,0.692559,...,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Laborers,Business Entity Type 3,reg oper account,block of flats,Panel,No
291079,13788,327.0,4632,NaN,2,2,0,NaN,0.690533,0.506484,...,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,Housing,reg oper spec account,block of flats,Panel,No
286592,17599,844.0,1135,NaN,3,3,1,NaN,0.388123,NaN,...,Working,Secondary / secondary special,Married,House / apartment,NaN,Business Entity Type 3,NaN,NaN,NaN,NaN
153384,22608,NaN,4947,NaN,2,2,0,NaN,0.679831,0.763640,...,Pensioner,Secondary / secondary special,Married,House / apartment,NaN,XNA,reg oper account,block of flats,"Stone, brick",No
130483,13321,5887.0,5989,NaN,2,2,0,0.622110,0.447977,0.508287,...,Working,Higher education,Separated,House / apartment,Core staff,School,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178573,23033,222.0,3995,NaN,2,2,0,NaN,0.772702,0.621226,...,Working,Secondary / secondary special,Separated,House / apartment,High skill tech staff,Transport: type 4,NaN,block of flats,Panel,No
31963,10854,2412.0,2908,15.0,2,2,0,0.236655,0.515272,0.553165,...,Working,Secondary / secondary special,Married,House / apartment,Laborers,Business Entity Type 3,org spec account,block of flats,"Stone, brick",No
220065,10453,1866.0,2169,NaN,2,2,0,0.591807,0.603728,0.612704,...,Commercial associate,Secondary / secondary special,Married,House / apartment,Core staff,Business Entity Type 2,NaN,NaN,NaN,NaN
285924,10267,2593.0,2345,NaN,2,2,0,NaN,0.668714,0.104795,...,Working,Secondary / secondary special,Single / not married,With parents,Sales staff,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No


### Fitur Kategorik

In [55]:
X_train_fix_cat=X_train_fix[cat]

In [56]:
data = {
    'fitur': X_train_fix_cat.columns,
    'nuniq': X_train_fix_cat.nunique(),
    'uniq': X_train_fix_cat.apply(lambda x: x.unique())
}
X_cat_uniq = pd.DataFrame(data)

X_cat_uniq

,fitur,nuniq,uniq
NAME_CONTRACT_TYPE,NAME_CONTRACT_TYPE,2,"[Cash loans, Revolving loans]"
CODE_GENDER,CODE_GENDER,2,"[F, M]"
FLAG_OWN_CAR,FLAG_OWN_CAR,2,"[N, Y]"
FLAG_OWN_REALTY,FLAG_OWN_REALTY,2,"[Y, N]"
NAME_TYPE_SUITE,NAME_TYPE_SUITE,7,"[Unaccompanied, Family, Spouse, partner, Child..."
NAME_INCOME_TYPE,NAME_INCOME_TYPE,8,"[Commercial associate, Working, Pensioner, Sta..."
NAME_EDUCATION_TYPE,NAME_EDUCATION_TYPE,5,"[Secondary / secondary special, Higher educati..."
NAME_FAMILY_STATUS,NAME_FAMILY_STATUS,6,"[Single / not married, Married, Separated, Civ..."
NAME_HOUSING_TYPE,NAME_HOUSING_TYPE,6,"[House / apartment, Municipal apartment, Rente..."
OCCUPATION_TYPE,OCCUPATION_TYPE,18,"[Laborers, nan, Core staff, High skill tech st..."


In [57]:
X_train_fix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246004 entries, 227344 to 37137
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   DAYS_BIRTH                   246004 non-null  int64  
 1   DAYS_EMPLOYED                201599 non-null  float64
 2   DAYS_ID_PUBLISH              246004 non-null  int64  
 3   OWN_CAR_AGE                  83845 non-null   float64
 4   REGION_RATING_CLIENT         246004 non-null  int64  
 5   REGION_RATING_CLIENT_W_CITY  246004 non-null  int64  
 6   REG_CITY_NOT_WORK_CITY       246004 non-null  int64  
 7   EXT_SOURCE_1                 107252 non-null  float64
 8   EXT_SOURCE_2                 245454 non-null  float64
 9   EXT_SOURCE_3                 197193 non-null  float64
 10  DAYS_LAST_PHONE_CHANGE       246003 non-null  float64
 11  NAME_CONTRACT_TYPE           246004 non-null  object 
 12  CODE_GENDER                  246004 non-null  object 


In [58]:
cat_lab=X_cat_uniq['fitur'][X_cat_uniq["nuniq"]==2].tolist()

In [59]:
cat_nlab=[col for col in cat if col not in cat_lab]

In [60]:
cat_ord=['NAME_EDUCATION_TYPE']
cat_nom=[col for col in cat_nlab if col not in cat_ord]

In [61]:
X_train_fix['NAME_EDUCATION_TYPE'].unique()

array(['Secondary / secondary special', 'Higher education',
       'Lower secondary', 'Incomplete higher', 'Academic degree'],
      dtype=object)

In [62]:
cat_lab

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'EMERGENCYSTATE_MODE']

In [63]:
cat_nom

['NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE']

### Fitur Numerik - Outlier

In [64]:
import pandas as pd
import scipy.stats as stats

# Hitung skewness dan kurtosis untuk setiap kolom
kolom = X_train[num].columns
skewness = X_train[num].skew()
kurtosis = X_train[num].kurtosis()

# Buat DataFrame baru dengan kolom nama, skewness, dan kurtosis
data = {
    'Nama Kolom': kolom,
    'Skewness': skewness,
    'Kurtosis': kurtosis
}
df_stats = pd.DataFrame(data)

# Tampilkan DataFrame dengan nilai skewness dan kurtosis
print(df_stats)

                                              Nama Kolom  Skewness  Kurtosis
DAYS_BIRTH                                    DAYS_BIRTH  0.115423 -1.048660
DAYS_EMPLOYED                              DAYS_EMPLOYED  1.967045  4.743147
DAYS_ID_PUBLISH                          DAYS_ID_PUBLISH -0.352391 -1.104731
OWN_CAR_AGE                                  OWN_CAR_AGE  2.753827  9.295969
REGION_RATING_CLIENT                REGION_RATING_CLIENT  0.087798  0.799462
REGION_RATING_CLIENT_W_CITY  REGION_RATING_CLIENT_W_CITY  0.060084  0.933719
REG_CITY_NOT_WORK_CITY            REG_CITY_NOT_WORK_CITY  1.281514 -0.357725
EXT_SOURCE_1                                EXT_SOURCE_1 -0.066800 -0.966387
EXT_SOURCE_2                                EXT_SOURCE_2 -0.794133 -0.265433
EXT_SOURCE_3                                EXT_SOURCE_3 -0.410778 -0.663707
DAYS_LAST_PHONE_CHANGE            DAYS_LAST_PHONE_CHANGE  0.712562 -0.309739


In [65]:
filtered_df = df_stats[(df_stats['Skewness'].abs() > 0.5)]
filtered_df = filtered_df[['Nama Kolom','Skewness']]
print(filtered_df)

                                    Nama Kolom  Skewness
DAYS_EMPLOYED                    DAYS_EMPLOYED  1.967045
OWN_CAR_AGE                        OWN_CAR_AGE  2.753827
REG_CITY_NOT_WORK_CITY  REG_CITY_NOT_WORK_CITY  1.281514
EXT_SOURCE_2                      EXT_SOURCE_2 -0.794133
DAYS_LAST_PHONE_CHANGE  DAYS_LAST_PHONE_CHANGE  0.712562


In [66]:
num_skew=filtered_df['Nama Kolom'].tolist()
num_nor=[col for col in num if col not in num_skew]

In [67]:
#break

In [68]:
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

In [69]:
from scipy.stats.mstats import winsorize

##Pipeline

In [70]:
num_imputer_skew = SimpleImputer(strategy='median')

num_skew_pipeline = Pipeline([
    ('imputer', num_imputer_skew),
    ('winsorizer', Winsorizer(capping_method='gaussian', tail='both', fold=0.05)),
    ('scaler', RobustScaler())
])

num_imputer_nor = SimpleImputer(strategy='mean')

num_nor_pipeline = Pipeline([
    ('imputer', num_imputer_nor),
    ('scaler', StandardScaler())
])

cat_imputer = SimpleImputer(strategy='most_frequent')

cat_nom_pipeline = Pipeline([
    ('imputer', cat_imputer),
    ('encoder', OneHotEncoder())
])

cat_lab_pipeline = Pipeline([
    ('imputer', cat_imputer),
    ('encoder', OrdinalEncoder())
])

cat_ord_pipeline = Pipeline([
    ('encoder', OrdinalEncoder(categories=[[ 'Lower secondary',
                                            'Secondary / secondary special',
                                             'Incomplete higher',
                                            'Higher education',
                                            'Academic degree']]))
])



pipeline = ColumnTransformer(
    transformers=[
        ('num_skew', num_skew_pipeline, num_skew),
        ('num_nor', num_nor_pipeline, num_nor),
        ('cat_nom', cat_nom_pipeline, cat_nom),
        ('cat_lab', cat_lab_pipeline, cat_lab),
        ('cat_ord', cat_ord_pipeline, cat_ord)
    ]
)

In [71]:
X_train_fix.shape

(246004, 26)

In [72]:
X_train_scaled = pipeline.fit_transform(X_train_fix)
X_test_scaled = pipeline.transform(X_test)

In [73]:
X_train_scaled.shape

(246004, 134)

In [74]:
#break

## TSVD

### Mencari nilai n_estimator

kode berasal dari
https://www.kaggle.com/code/rahulpatel11315/selecting-the-best-number-of-components-for-tsvd

In [75]:
from sklearn.decomposition import TruncatedSVD

In [76]:
X_train_scaled.shape

(246004, 134)

In [77]:
tsvd = TruncatedSVD(n_components=X_train_scaled.shape[1]-1)
X_tsvd = tsvd.fit(X_train_scaled)

In [78]:
tsvd_var_ratios = tsvd.explained_variance_ratio_

In [79]:
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0

     # Set initial number of features
    n_components = 0

    # For the explained variance of each feature:
    for explained_variance in var_ratio:

        # Add the explained variance to the total
        total_variance += explained_variance

        # Add one to the number of components
        n_components += 1

        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
           break

    # Return the number of components
    return n_components

In [80]:
n_components=select_n_components(tsvd_var_ratios, 0.95)

In [81]:
n_components

41

### Reduksi dengan TSVD

In [82]:
svd = TruncatedSVD(n_components)  # Specify the desired number of components

X_train_svd = svd.fit_transform(X_train_scaled)
X_test_svd = svd.transform(X_test_scaled)

In [83]:
var_explained = svd.explained_variance_ratio_.sum()
var_explained

0.951970339412625

In [84]:
X_train_svd.shape

(246004, 41)

In [85]:
X_train_svd.shape

(246004, 41)

In [86]:
X_test_svd.shape

(61502, 41)

## Resampling

In [87]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [88]:
#ros = SMOTE(sampling_strategy=
#            'auto'
#            #{1: (123002)}
#            ,
#            random_state=42)

rus = RandomUnderSampler(sampling_strategy=
                         'auto'
                        #{0: (123002)}
                         ,
                         random_state=42)

In [89]:
# Oversampling
#X_oversampled, y_oversampled = ros.fit_resample(X_train_svd, y_train)

In [90]:
#y_oversampled.value_counts()

In [91]:
# Undersampling
#X_resampled, y_resampled = rus.fit_resample(X_oversampled, y_oversampled)

In [92]:
X_resampled, y_resampled = rus.fit_resample(X_train_svd, y_train)

In [93]:
y_resampled.value_counts()

0    19772
1    19772
Name: TARGET, dtype: int64

In [94]:
y_resampled.value_counts()

0    19772
1    19772
Name: TARGET, dtype: int64

In [95]:
X_train_fin=X_resampled
y_train_fin=y_resampled


In [96]:
X_train_fin

array([[ 3.71450071e+00,  1.88711291e-01,  4.32910734e-01, ...,
         4.33362847e-02, -7.19068528e-02, -9.22430800e-02],
       [ 2.66504714e+00, -8.73635559e-02,  8.60373256e-01, ...,
         3.12428044e-01, -5.40084371e-02, -1.23885185e-01],
       [ 3.20328194e+00, -2.72345866e+00, -3.08448507e-01, ...,
         2.94367214e-01,  3.54581426e-02, -1.07674826e-02],
       ...,
       [ 2.63049386e+00, -2.95781783e-01,  8.08352019e-01, ...,
        -1.03997638e-01,  2.53764556e-02, -2.98753072e-03],
       [ 2.13970585e+00,  1.54348153e-02,  9.75264160e-01, ...,
         2.36505740e-01, -3.47662796e-02,  5.72636661e-02],
       [ 2.13037740e+00, -2.18883790e-01,  2.01224686e+00, ...,
         4.43229911e-02, -1.17732896e-02, -4.55545115e-03]])

In [97]:
#break

# Inisiasi Model

In [98]:
lr = LogisticRegression()

In [99]:
lgb =LGBMClassifier()

In [100]:
xgb=XGBClassifier()

In [101]:
def evaluasi (model):
  y_train_pred = model.predict(X_train_fin)
  y_pred = model.predict(X_test_svd)

  print('Train')
  print(classification_report(y_train_fin, y_train_pred))
  print('Test')
  print(classification_report(y_test, y_pred))
  print('Train')
  print(confusion_matrix(y_train_fin, y_train_pred))
  print('Test')
  print(confusion_matrix(y_test, y_pred))
  print('Train')
  print(roc_auc_score(y_train_fin, y_train_pred))
  print('Test')
  print(roc_auc_score(y_test, y_pred))

In [102]:
#break

# Training dan Evaluasi

## Log Reg

In [103]:
model_lr=lr.fit(X_train_fin, y_train_fin)

In [104]:
evaluasi (model_lr)

Train
              precision    recall  f1-score   support

           0       0.67      0.68      0.67     19772
           1       0.67      0.67      0.67     19772

    accuracy                           0.67     39544
   macro avg       0.67      0.67      0.67     39544
weighted avg       0.67      0.67      0.67     39544

Test
              precision    recall  f1-score   support

           0       0.96      0.67      0.79     56450
           1       0.15      0.67      0.25      5052

    accuracy                           0.67     61502
   macro avg       0.56      0.67      0.52     61502
weighted avg       0.89      0.67      0.74     61502

Train
[[13356  6416]
 [ 6483 13289]]
Test
[[37776 18674]
 [ 1668  3384]]
Train
0.6738063928788185
Test
0.6695138530934613


## LGB

In [105]:
model_lgb=lgb.fit(X_train_fin, y_train_fin)

In [106]:
evaluasi (model_lgb)

Train
              precision    recall  f1-score   support

           0       0.73      0.72      0.73     19772
           1       0.73      0.74      0.73     19772

    accuracy                           0.73     39544
   macro avg       0.73      0.73      0.73     39544
weighted avg       0.73      0.73      0.73     39544

Test
              precision    recall  f1-score   support

           0       0.96      0.66      0.78     56450
           1       0.15      0.68      0.25      5052

    accuracy                           0.66     61502
   macro avg       0.56      0.67      0.51     61502
weighted avg       0.89      0.66      0.74     61502

Train
[[14316  5456]
 [ 5239 14533]]
Test
[[37206 19244]
 [ 1607  3445]]
Train
0.7295417762492413
Test
0.670502350400827


## XGB

In [107]:
model_xgb=xgb.fit(X_train_fin, y_train_fin)

In [108]:
evaluasi (model_xgb)

Train
              precision    recall  f1-score   support

           0       0.81      0.80      0.80     19772
           1       0.80      0.81      0.81     19772

    accuracy                           0.80     39544
   macro avg       0.80      0.80      0.80     39544
weighted avg       0.80      0.80      0.80     39544

Test
              precision    recall  f1-score   support

           0       0.96      0.65      0.77     56450
           1       0.15      0.67      0.24      5052

    accuracy                           0.65     61502
   macro avg       0.55      0.66      0.51     61502
weighted avg       0.89      0.65      0.73     61502

Train
[[15858  3914]
 [ 3805 15967]]
Test
[[36746 19704]
 [ 1691  3361]]
Train
0.8047997167711916
Test
0.658114409082653


In [109]:
#break

## Log Reg - RandomSearch

In [110]:
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier

param_grid = {'C': [0.1, 1.0, 10.0],
              'penalty': ['l1', 'l2'],
              'max_iter': [100,1000,10000],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag','saga']}



random_search = RandomizedSearchCV(model_lr, param_grid, n_iter=3, scoring='roc_auc', cv=5, random_state=42)


random_search.fit(X_train_fin, y_train_fin)


print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)


Best parameters:  {'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 10000, 'C': 0.1}
Best score:  0.7331827055532207


In [111]:
best_model_lr = random_search.best_estimator_

In [112]:
evaluasi (best_model_lr)

Train
              precision    recall  f1-score   support

           0       0.67      0.68      0.67     19772
           1       0.67      0.67      0.67     19772

    accuracy                           0.67     39544
   macro avg       0.67      0.67      0.67     39544
weighted avg       0.67      0.67      0.67     39544

Test
              precision    recall  f1-score   support

           0       0.96      0.67      0.79     56450
           1       0.15      0.67      0.25      5052

    accuracy                           0.67     61502
   macro avg       0.56      0.67      0.52     61502
weighted avg       0.89      0.67      0.74     61502

Train
[[13350  6422]
 [ 6483 13289]]
Test
[[37751 18699]
 [ 1663  3389]]
Train
0.6736546631600242
Test
0.6697872717186785


## LGB - RandomSearch

In [113]:

param_grid = {
    'boosting_type': ['gbdt', 'dart', 'goss', 'rf'],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [-1, 1, 3, 5, 7, 9, 11]
}

model = LGBMClassifier()

random_search = RandomizedSearchCV(model, param_grid, n_iter=3, scoring='roc_auc', cv=5, random_state=42)

random_search.fit(X_train_fin, y_train_fin)

print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)


Best parameters:  {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'boosting_type': 'gbdt'}
Best score:  0.7287408727641382


In [114]:
best_model_rs = random_search.best_estimator_

In [115]:
evaluasi (best_model_rs)

Train
              precision    recall  f1-score   support

           0       0.69      0.67      0.68     19772
           1       0.68      0.70      0.69     19772

    accuracy                           0.68     39544
   macro avg       0.68      0.68      0.68     39544
weighted avg       0.68      0.68      0.68     39544

Test
              precision    recall  f1-score   support

           0       0.96      0.65      0.77     56450
           1       0.15      0.69      0.24      5052

    accuracy                           0.65     61502
   macro avg       0.55      0.67      0.51     61502
weighted avg       0.89      0.65      0.73     61502

Train
[[13272  6500]
 [ 5969 13803]]
Test
[[36699 19751]
 [ 1590  3462]]
Train
0.6846803560590734
Test
0.6676941526459628


## XGB - RandomSearch

In [116]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.3, 0.1 , 0.01, 0.001],
    'max_depth': [3, 5, 6, 7, 9]
}

model = XGBClassifier()

random_search = RandomizedSearchCV(model_xgb, param_grid, n_iter=3, scoring='roc_auc', cv=5, random_state=42)

random_search.fit(X_train_fin, y_train_fin)

print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)


Best parameters:  {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.3}
Best score:  0.7253227897350781


In [117]:
best_model_xgb = random_search.best_estimator_

In [118]:
evaluasi (best_model_xgb)

Train
              precision    recall  f1-score   support

           0       0.70      0.69      0.70     19772
           1       0.70      0.71      0.70     19772

    accuracy                           0.70     39544
   macro avg       0.70      0.70      0.70     39544
weighted avg       0.70      0.70      0.70     39544

Test
              precision    recall  f1-score   support

           0       0.96      0.66      0.78     56450
           1       0.15      0.68      0.25      5052

    accuracy                           0.66     61502
   macro avg       0.55      0.67      0.51     61502
weighted avg       0.89      0.66      0.74     61502

Train
[[13741  6031]
 [ 5761 14011]]
Test
[[37049 19401]
 [ 1622  3430]]
Train
0.7018005259963584
Test
0.6676271786704367


# Simpan Model

In [119]:
best_model = model_lgb

In [148]:
# Save Pipeline
with open('pipeline.pkl', 'wb') as file_1:
  pickle.dump(pipeline, file_1)

# model
with open('TSDV.pkl', 'wb') as file_2:
  pickle.dump(svd, file_2)

# model
with open('best_model.pkl', 'wb') as file_3:
  pickle.dump(best_model, file_3)

# list fs
with open('kolom_fitur.txt', 'w') as file_4:
  json.dump(kolom_fitur, file_4)

# Data Inference

In [121]:
df_inf=df2[kolom_fitur]

In [122]:
df_inf_scaled = pipeline.transform(df_inf)

In [123]:
df_inf_svd = svd.transform(df_inf_scaled)

In [124]:
inf_pred_proba = best_model.predict_proba(df_inf_svd)

inf_pred_proba

array([[0.75618762, 0.24381238],
       [0.67109945, 0.32890055],
       [0.6941261 , 0.3058739 ],
       ...,
       [0.71688777, 0.28311223],
       [0.59042358, 0.40957642],
       [0.6852427 , 0.3147573 ]])

In [125]:
inf_pred = best_model.predict(df_inf_svd)

inf_pred

array([0, 0, 0, ..., 0, 0, 0])

In [126]:
df_inf_result = pd.DataFrame(inf_pred_proba, columns=['proba_0', 'proba_1'])
df_inf_result['inf_pred'] = inf_pred

df_inf_result

,proba_0,proba_1,inf_pred
0,0.756188,0.243812,0
1,0.671099,0.328901,0
2,0.694126,0.305874,0
3,0.727013,0.272987,0
4,0.631394,0.368606,0
...,...,...,...
48739,0.540631,0.459369,0
48740,0.734878,0.265122,0
48741,0.716888,0.283112,0
48742,0.590424,0.409576,0


In [127]:
df_inf_result = df_inf_result.join(df2['SK_ID_CURR'])

In [128]:
df_inf_result

,proba_0,proba_1,inf_pred,SK_ID_CURR
0,0.756188,0.243812,0,100001
1,0.671099,0.328901,0,100005
2,0.694126,0.305874,0,100013
3,0.727013,0.272987,0,100028
4,0.631394,0.368606,0,100038
...,...,...,...,...
48739,0.540631,0.459369,0,456221
48740,0.734878,0.265122,0,456222
48741,0.716888,0.283112,0,456223
48742,0.590424,0.409576,0,456224


In [149]:
df_inf_result.to_csv('result.csv', index=False)

# KESIMPULAN

## FE
- Pada FE kita menghapus kolom yang memiliki persentase nilai missing value di atas 40%, hal ini dilakukan agar fitur pada data kita kita tidak banyak data sintetik
- Fitur Selection dilakukan dengan mengambil fitur kategorik yang memiliki p-value > 0.05 pada pencarian nilai chi-vakue dan mengambil nilai korelasi >|+/-0.5|
- Fitur numerik dan kategorik dibagi menjadi :
  - Numerik, dibagi menjadi numerik skew (nilai skew > |+/-0.5|) dan normal (nilai skew < |+/-0.5|). numerik skew dilakukan simpler imputer nilai median, handling outlier dengan winsorizer, dan Robust Scaler sedangkan numerik normal dilakukan simpler imputer nilai mean dan Standard Scaler
  - Kategorik, dibagi menjadi tiga: ordinal, nominal, dan label. Pada ordinal dan label dilakukan Ordinal Encoding, dan nominal dilakukan One Hot Encoding
- Dilakukan Fitur Reduction dengan TruncatedSVD karena fitur kita memiliki matrix yang sparse.
- Dilakukan Resampling dengan metode Undersampling pada kelas mayoritas (kelas 0). Hal ini dilakukan agar kelas 0 dan kelas 1 memiliki jumlah data yang sama. Kita tidak melakukan Oversampling karena kita tidak menginginkan data sintetik pada kelas minoritas

## Training
- Kita menggunakan LogReg, LGB, dan XGB untuk mencari nilai evaluasi ROC-AUC yang bagus. Algoritma tersebut dipilih karena memiliki kecepatan yang baik dan cenderung tahan terhadap data robust (LGB dan XGB)

## Evaluasi
- Berdasarkan Nilai ROC-AUC pada data test maka model yang bagus adalah model yang menggunakan algoritma Light Gradien Boosting dengan nilai ROC-AUC 67%

- Model ini belum lah bagus jika melihat dari nilai ROC-AUC. Kita bisa meningkatkan model kita dengan mencoba algoritma klasifikasi yang lain dan/atau FE yang lebih baik